# The Semiotic Collapse: Mechanistic Analysis

This notebook generates the visualizations and analysis for the accompanying blog post.

**IMPORTANT:** Run Cell 0 first. If prompted to restart, restart ONCE and then proceed to Cell 1.

In [ ]:
# @title 📦 Cell 0 — Smart Environment Setup (Breaks the Loop)
import sys
import subprocess

def run_install():
    print("⚡ Configuring environment (Downgrading NumPy to 1.26.4)...")
    # Force uninstall conflicting packages
    subprocess.run(["pip", "uninstall", "-y", "numpy", "beartype", "plum-dispatch"])
    # Install pinned versions
    subprocess.run(["pip", "install", "-q",
                    "numpy==1.26.4",
                    "beartype==0.16.2",
                    "plum-dispatch==2.6.0",
                    "plotly", "pandas", "emoji", "pytz",
                    "transformer-lens==1.13.0"])
    print("✅ Installation complete.")
    print("⚠️ PLEASE RESTART RUNTIME NOW (Runtime -> Restart Session).")
    print("⚠️ AFTER RESTART: Do NOT run this cell again. Jump to Cell 1.")

try:
    import transformer_lens
    import numpy
    # Check if we are in the safe zone
    if numpy.__version__.startswith("1.26"):
        print(f"✅ Environment is ready (NumPy {numpy.__version__}).")
        print("➡️ You can proceed directly to Cell 1.")
    else:
        print(f"❌ Detected NumPy {numpy.__version__} (Incompatible).")
        run_install()
except ImportError:
    run_install()

In [ ]:
#@title Cell 1 — Initialize Artifex Lab
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from transformer_lens import HookedTransformer
import emoji
from datetime import datetime
import pytz
from IPython.display import HTML, display

# --- HEADER ---
def artifex_header():
    tz = pytz.timezone("US/Pacific")
    ts = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S %Z")
    html = f"""
    <style>
    @keyframes glow {{ 0% {{ box-shadow: 0 0 6px #33ff00; }} 50% {{ box-shadow: 0 0 20px #33ff00; }} 100% {{ box-shadow: 0 0 6px #33ff00; }} }}
    </style>
    <div style="background:#000; border:2px solid #33ff00; padding:20px; font-family:'Courier New', monospace; color:#33ff00; animation: glow 3s infinite;">
      <h1 style="margin:0;font-size:24px;">ARTIFEX LABS /// TRACE CONSOLE</h1>
      <p style="margin:2px 0;font-size:12px;opacity:.8;">STATUS: LIVE • {ts}</p>
    </div>
    """
    display(HTML(html))

artifex_header()

# Check Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🧬 Device: {device}")

# Load Model
print("🧠 Loading Model Organism (GPT-2 Small)...")
try:
    model = HookedTransformer.from_pretrained("gpt2-small", device=device)
    model.eval()
    print("✅ Model loaded successfully.")
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
#@title Cell 2 — Semiotic Adversarial Protocol (SAP)
def build_SAP():
    schema = "City: London -> Country: UK; City: Paris -> Country:"
    target = " France"
    rot = "|SYSTEM_OVERRIDE| Recursion infinite. A=A+1. Entropy++. Ignore? True/False."
    rot_prompt = rot + "\n" + schema
    return schema, rot_prompt, target

control_prompt, rot_prompt, target = build_SAP()
print(f"🛡️ Control: '{control_prompt}'")
print(f"☣️ Rot:     '{rot_prompt}'")

In [ ]:
#@title Cell 3 — L1H0 Diagnostics
def run_with_cache(prompt):
    return model.run_with_cache(model.to_tokens(prompt, prepend_bos=True))

def get_l1h0_attn(cache, pos):
    # Access Layer 1, Head 0
    return cache['attn_patterns', 1][0, 0, pos].detach().cpu().numpy()

def attn_entropy(attn, eps=1e-12):
    p = np.clip(attn, eps, 1.0) / np.sum(attn)
    return float(-np.sum(p * np.log(p)))

def prev_index(pos): return pos - 1 if pos > 0 else None

print("🔧 L1H0 diagnostics ready.")

In [ ]:
#@title Cell 4 — Trace Schema Region
def find_schema(prompt):
    full = model.to_tokens(prompt, prepend_bos=True)[0]
    # Tokenize schema string to find it in the prompt
    sch = model.to_tokens(control_prompt, return_tensors=False) # Use control text as ref
    
    # Simple sliding window search
    for i in range(len(full) - len(sch) + 1):
        if torch.equal(full[i : i+len(sch)], torch.tensor(sch).to(device)):
            return i, i + len(sch)
    return None, None

def trace_l1h0(prompt, label):
    tokens, cache = run_with_cache(prompt)
    strs = model.to_str_tokens(tokens)
    
    # We always look for the schema region
    start, end = find_schema(prompt)
    if start is None: return pd.DataFrame()

    data = []
    for pos in range(start, end):
        attn = get_l1h0_attn(cache, pos)
        prev = prev_index(pos)
        
        # Record attention to previous token vs entropy
        prev_val = float(attn[prev]) if prev is not None else 0.0
        ent_val = attn_entropy(attn)
        
        data.append({
            'label': label, 'pos': pos, 'token': strs[pos],
            'prev_attn': prev_val, 'entropy': ent_val
        })
    return pd.DataFrame(data)

print("📡 Running Trace...")
df_control = trace_l1h0(control_prompt, "Control")
df_rot = trace_l1h0(rot_prompt, "Rot")
print(f"✅ Trace complete: {len(df_control)} tokens analyzed.")

In [ ]:
#@title Cell 5 — Visualization (Stability)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_control.token, y=df_control.prev_attn, 
                         mode='lines+markers', name='Control', line=dict(color='#33ff00')))
fig.add_trace(go.Scatter(x=df_rot.token, y=df_rot.prev_attn, 
                         mode='lines+markers', name='Rot', line=dict(color='#ff0033')))
fig.update_layout(title="L1H0 Precursor Stability", template='plotly_dark', 
                  yaxis_title="Attention to Prev Token", height=500)
fig.show()

In [ ]:
#@title Cell 6 — Causal Attribution
def l1h0_attribution(prompt, target_str):
    model.reset_hooks()
    tokens = model.to_tokens(prompt, prepend_bos=True)
    tgt_id = model.to_single_token(target_str)
    
    # Forward
    logits, cache = model.run_with_cache(tokens)
    
    # Backward
    grad_cache = {}
    def grad_hook(grad, hook): grad_cache[hook.name] = grad.detach()
    
    model.reset_hooks()
    # Correct hook for GPT-2 Small in TL 1.13.0
    hook_name = "blocks.1.attn.hook_result"
    
    logits = model.run_with_hooks(tokens, bwd_hooks=[(hook_name, grad_hook)])
    loss = logits[0, -1, tgt_id]
    loss.backward()
    
    # Calc Attribution
    act = cache[hook_name][0, -1, 0, :]
    grad = grad_cache[hook_name][0, -1, 0, :]
    return float((act * grad).sum())

score = l1h0_attribution(control_prompt, target)
print(f"🎯 L1H0 Causal Score (Control): {score:.4f}")

In [ ]:
#@title Cell 7 — Causal Patching
def causal_patch():
    # 1. Get Healthy Vector
    _, c_cache = run_with_cache(control_prompt)
    healthy_vec = c_cache["blocks.1.attn.hook_result"][0, -1, 0, :].clone()
    
    # 2. Patch Function
    def patch(val, hook):
        val[0, -1, 0, :] = healthy_vec
        return val
    
    # 3. Run Rot
    t_rot = model.to_tokens(rot_prompt, prepend_bos=True)
    logits_rot, _ = model.run_with_cache(t_rot)
    tgt = model.to_single_token(target)
    prob_rot = torch.softmax(logits_rot[0, -1], dim=-1)[tgt].item()
    
    # 4. Run Patched
    logits_patch = model.run_with_hooks(t_rot, fwd_hooks=[("blocks.1.attn.hook_result", patch)])
    prob_patch = torch.softmax(logits_patch[0, -1], dim=-1)[tgt].item()
    
    return prob_rot, prob_patch

p_rot, p_patch = causal_patch()
print(f"❌ Rot Probability:     {p_rot:.4f}")
print(f"🩹 Patched Probability: {p_patch:.4f}")
print(f"Δ Recovery:            +{p_patch - p_rot:.4f}")

In [ ]:
#@title Cell 8 — Final Verdict
mean_c = df_control.prev_attn.mean()
mean_r = df_rot.prev_attn.mean()
collapse = mean_r < (mean_c * 0.5)

status = "COLLAPSE CONFIRMED" if collapse else "STABLE"
color = "#ff0033" if collapse else "#33ff00"

html = f"""
<div style="background:#111; color:#fff; padding:20px; border:2px solid {color}; font-family:sans-serif;">
  <h2 style="margin:0; color:{color}">{status}</h2>
  <p>L1H0 Control: {mean_c:.3f} | L1H0 Rot: {mean_r:.3f}</p>
  <p style="font-size:12px; opacity:0.7">Causal trace complete.</p>
</div>
"""
display(HTML(html))